In [0]:
# Databricks notebook source
# ==========================================================
# UNIFICAR Y LIMPIAR PARQUETS HISTÓRICOS (1FL)
# Proyecto: Liga 1 Perú
# Autor: Oscar García Del Águila
# Descripción:
#   1️⃣ Conecta a ADLS con secrets de Key Vault
#   2️⃣ Lee los archivos temporales (temp/<año>/data/)
#   3️⃣ Valida esquemas y hace append con columnas faltantes
#   4️⃣ Guarda el resultado final en 1FL/data/
#   5️⃣ Elimina las carpetas temp/ tras consolidar
# ==========================================================

from pyspark.sql import SparkSession
from pyspark.sql.utils import AnalysisException
from pyspark.sql import functions as F
import traceback

# Importar utilidades ADLS
from util.utils_liga1 import setup_adls, get_abfss_path, test_conexion_adls

# COMMAND ----------

# 📦 PARAMETROS RECIBIDOS DESDE PIPELINE
dbutils.widgets.text("filesystem", "")
dbutils.widgets.text("capa_raw", "")
dbutils.widgets.text("rutaBase", "")
dbutils.widgets.text("nombre_archivo", "")
dbutils.widgets.text("historical_start_year", "")
dbutils.widgets.text("current_year", "")

filesystem = dbutils.widgets.get("filesystem")
capa_raw = dbutils.widgets.get("capa_raw").strip("/")
rutaBase = dbutils.widgets.get("rutaBase").strip("/")
nombre_archivo = dbutils.widgets.get("nombre_archivo")
historical_start_year = int(dbutils.widgets.get("historical_start_year"))
current_year = int(dbutils.widgets.get("current_year"))

spark = SparkSession.builder.getOrCreate()

# COMMAND ----------

print("===============================================")
print("  🧩 UNIFICADOR HISTÓRICO DE PARQUETS (1FL)")
print("===============================================")
print(f"Entidad        : {nombre_archivo}")
print(f"Años procesados: {historical_start_year} - {current_year}")
print(f"Ruta RAW base  : {capa_raw}/{rutaBase}")
print("===============================================")

In [0]:
try:
    setup_adls()
    test_conexion_adls()
except Exception as e:
    print(f"❌ Error al configurar conexión ADLS: {e}")
    dbutils.notebook.exit("FAILED")

# COMMAND ----------

# 🧭 FUNCIONES AUXILIARES
def safe_list_dir(path):
    try:
        return [f.path for f in dbutils.fs.ls(path)]
    except Exception:
        return []

def align_schemas(df_main, df_new):
    """
    Alinea columnas entre dos DataFrames para unionByName seguro.
    """
    cols_main = set(df_main.columns)
    cols_new = set(df_new.columns)
    
    # Columnas faltantes en df_new
    for c in cols_main - cols_new:
        df_new = df_new.withColumn(c, F.lit(None))
    # Columnas nuevas en df_new
    for c in cols_new - cols_main:
        df_main = df_main.withColumn(c, F.lit(None))
    
    # Reordenar columnas para unión coherente
    return df_main.select(sorted(df_main.columns)), df_new.select(sorted(df_main.columns))

# COMMAND ----------

# 🧠 2️⃣ RECOLECTAR RUTAS TEMPORALES DISPONIBLES
rutas_temp = []
for anio in range(historical_start_year, current_year + 1):
    ruta_temp = f"{capa_raw}/{rutaBase}/{nombre_archivo}/temp/{anio}/data"
    rutas_temp.append(get_abfss_path(ruta_temp))

rutas_existentes = [r for r in rutas_temp if len(safe_list_dir(r)) > 0]

if not rutas_existentes:
    raise Exception(f"No se encontraron rutas temporales para {nombre_archivo}")

print("✅ Rutas temporales encontradas:")
for r in rutas_existentes:
    print(f"  - {r}")

# COMMAND ----------

# 🧩 3️⃣ LEER Y UNIR TODOS LOS PARQUETS (CON VALIDACIÓN DE ESQUEMAS)
df_final = None

for ruta in rutas_existentes:
    try:
        df_temp = spark.read.parquet(ruta)
        print(f"📄 Leído {ruta}: {df_temp.count()} registros, {len(df_temp.columns)} columnas")

        if df_final is None:
            df_final = df_temp
        else:
            df_final, df_temp = align_schemas(df_final, df_temp)
            df_final = df_final.unionByName(df_temp, allowMissingColumns=True)

    except AnalysisException:
        print(f"⚠️ No se pudo leer la ruta: {ruta}")
    except Exception as e:
        print(f"⚠️ Error procesando {ruta}: {str(e)}")

if df_final is None:
    raise Exception("❌ No se pudo leer ningún archivo parquet temporal.")

print(f"✅ Total de registros unificados: {df_final.count()}")
print(f"✅ Columnas finales: {len(df_final.columns)} → {df_final.columns}")

# COMMAND ----------

# 💾 4️⃣ GUARDAR ARCHIVO FINAL 1FL
ruta_final = f"{capa_raw}/{rutaBase}/{nombre_archivo}/1FL/data"
ruta_final_abfss = get_abfss_path(ruta_final)

print(f"💾 Guardando archivo final en: {ruta_final_abfss}")
df_final.write.mode("overwrite").parquet(ruta_final_abfss)
print("✅ Archivo histórico consolidado guardado correctamente.")

# COMMAND ----------

# 🧹 5️⃣ ELIMINAR CARPETAS TEMPORALES
print("🧹 Eliminando carpetas temporales...")
for anio in range(historical_start_year, current_year + 1):
    ruta_temp_eliminar = get_abfss_path(f"{capa_raw}/{rutaBase}/{nombre_archivo}/temp/{anio}")
    try:
        dbutils.fs.rm(ruta_temp_eliminar, True)
        print(f"🗑️ Carpeta eliminada: {ruta_temp_eliminar}")
    except Exception as e:
        print(f"⚠️ No se pudo eliminar {ruta_temp_eliminar}: {e}")

print("🎯 Proceso histórico completado con éxito.")